In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import sys
sys.path.insert(1, './utils')

import torch
print(torch.cuda.get_device_name(torch.cuda.current_device()))
print('CUDA Version ' + torch.version.cuda)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import wandb
%env WANDB_SILENT=True
%env "WANDB_NOTEBOOK_NAME" "topml-sweep"

## Load data from file

In [ ]:
import dataparamsml
reload(dataparamsml)

dataDirectoryHome = '../gnnmodel'
scalingList = np.arange(10,21)/10
ieeecase = 'IEEE89'

dataParamsML = dataparamsml.dataParamsML(ieeecase, dataDirectoryHome, scalingList)

## Define funtions to train and test models

In [ ]:
import mlmodelwrapper
reload(mlmodelwrapper)
import testmlmodels
reload(testmlmodels)

In [ ]:
def defineModelTrainAndTest():    
    wandb.init(project=modelType.lower() + '-' + ieeecase.lower() + '-sweep')

    modelConfig = {}
    if(modelType.lower() == 'svm' or modelType.lower() == 'logr'):
        modelConfig['C'] = wandb.config.C
        modelConfig['max_iter'] = wandb.config.max_iter
    else:
        modelConfig['alpha'] = wandb.config.alpha
    
    allMlModels = mlmodelwrapper.mlmodelWrapper(dataParamsML, modelType.lower(), ieeecase, modelConfig)
    allMlModels.learnAndSaveAllMlModels()
    
    testMlModels = testmlmodels.testMlModels(allMlModels, dataParamsML)
    allBranchFailFreqs, allTrueFinalStates, allPredFinalStates = testMlModels.getAllFinalStates()
    
    allErrors = (allTrueFinalStates != allPredFinalStates)
    maxErrorPerLoading = np.max(np.mean(allErrors, axis=1)*100, axis=1)
    
    wandb.log({'average-error': np.mean(maxErrorPerLoading)})
    for ii, scalingValue in enumerate(scalingList):
        wandb.log({"average-error-{:0.2f}".format(scalingValue): maxErrorPerLoading[ii]})

## Run sweep

In [ ]:
# modelTypeList = ['SVM', 'MNB', 'CatNB', 'ComNB', 'BNB', 'LogR', 'RR']
modelTypeList = ['SVM', 'MNB']
for modelType in modelTypeList:
    print('Sweeping hyperparameters for ' + modelType + ' with ' + ieeecase + ' data...')
    if(modelType.lower() == 'svm' or modelType.lower() == 'logr'):
        sweep_configuration = {
            'method': 'bayes',
            'metric': 
            {
                'goal': 'minimize', 
                'name': 'average-error'
                },
            'parameters': 
            {
                'max_iter'   : {'values': [5000]},
                'C' : {'min': 1.00, 'max': 20.0},
             }
        }
    else:
        sweep_configuration = {
            'method': 'bayes',
            'metric': 
            {
                'goal': 'minimize', 
                'name': 'average-error'
                },
            'parameters': 
            {
                'alpha' : {'min': 0.01, 'max': 2.01},
             }
        }

    sweep_id = wandb.sweep(
        sweep=sweep_configuration, 
        project= modelType.lower() + '-' + ieeecase.lower() + '-sweep'
        )

    wandb.agent(sweep_id, function=defineModelTrainAndTest, count=2)
    wandb.finish()